# EEG Data and Data Processing

In this tutorial we will deal with the inspection and visualization of EEG-data via MNE and Braindecode. In particular, we are interested in
- how to **load preexisting data** with one of the two libraries
- how to **import** data **into "braindecode** format"
- which **data structures** are most important 
- how to **visualize** the gathered data
- how to **preprocess** the data

We will start with MNE, as Braindecode builds upon the raw data structure of MNE. However, below we summarize some knowledge that is essential for working with EEG-data

## EEG Basics
As we are going to work with EEG data we shouldn't omit the "theory". It is always useful to have some background knowledge in order to understand what is going on behind the scenes. Therefore, we want to get some insights on how EEG data is produced, how it is gathered and how we preprocess it. However, we will stay at the surface. If you want to dive deeper, you can consult the sources at the end.

### Electric Brain Signals
As said above, we briefly summarize the most basic facts about EEG. The first question that comes up is: Where is the EEG signal produced?

The first thing to understand is, that we can not measure the activities of single neurons. We are always measuring the electrical field which evolves as the sum of the contribution of 10000-50000 neurons, most often pyramidal ones. Hence, EEG is not well suited for locating single sources or understanding the behaviour of small scale structures in the brain. Also, measuring what is happening in the deeper brain structures is difficult.

To understand what is going on, we can take a look at the graphic below.
<div>
<img src="https://drive.google.com/uc?export=view&id=1P9h-Y_2u5AopUb0_Q8plDyUFv32Zt1NV" width="750"/>

<div text-align=center class="caption">Neuron (Source: https://cdn.kastatic.org/ka-perseus-images/3567fc3560de474001ec0dafb068170d30b0c751.png) </div>
</div>

As you can see, the neuron consists of several parts:
- Soma (Zellkörper): This is where all the important cell structures (nucleus, mitochondria ... ) 
- Dendrite: receives the ions from the "previous" cell 
- Axon hillock: serves as a reservoir that only "opens up" if enough current is collected
- Axon: the transmitter, like an electric cable
- Myelin: isolation material
- Ranvier's nodes (between the myelin parts): allows "jumping" of current for fast transmission
- Axon terminal with synapse: here, the electrical signal is converted into a chemical one

On a very high level, the current is flowing towards the synapse, ion canals open up and thus release neurotransmitters. This leads to a potential difference in the next neuron between dendrites and axons. When enough ions are transmitted, the current gets large enough and the electrical signal is transferred as a field from the axon hill towards the synapses where everything repeats. The difference between the dendrites and the synapses of the previous neuron is called an action potential. 

### The EEG Head
As we have some high school biology knowledge back in our heads, we can talk about where we measure the EEG signal. To understand how and which data we gather, we will look at the following picture.
<div>
<img src="https://drive.google.com/uc?export=view&id=17FEtTIxUQM7SLRAwjBnEXbDY1p6rSRB7" width="850"/>

<div text-align=center class="caption">EEG 10-20 System (Source: Wikipedia) </div>
</div>


The EEG-montage depicted here is set up according to the 10-20 system. That is, the electrodes are placed in a distance of 10 or 20 percent of either the front-back distance of the head or the left-right distance. The channel names are often used in agreement with the channel names depicted in the picture (or more). The naming follows a pattern briefly described in the following. 

At first, every latter stands for a position relative to the brain regions
- Fp: pre-frontal
- F: frontal (the **frontal lobe** contains the frontal cortex and is responsible for different things, among them planning of actions and expression of emotions)
- T: temporal (the **temporal lobe** contains the auditory cortex as well as structure for speech and memory)
- P: parietal (the **parietal lobe** is responsible e.g. for our spatial sense and navigation or the sense of touch) 
- O: Occipital (the **occipital lobe** is mainly responsible for our vision and contains most of the visual cortex)
- C: Central

In addition to that, we have numbers 1-8 and the letters "A" and "Z". The numbers can be related to the side of the head / the hemisphere of the brain. Odd numbers are located on the left side and even numbers on the right side of the head. In addition to the shown electrodes, one can use EOG-electrodes (electrooculogram) as well as ECG-electrodes (electrocardiogram) in order to measure the potentials due to our eye movements or our heart. There are also larger systems than the one shown above, however, it is sufficient for us to stick with the small one!

According to this, when using well-documented datasets, you can access and inspect the data at least on a basic level and understand where the signal comes from. 

### EEG Data
Now, we have to talk about the EEG data itself. Therefore, we will take a look at a graphic as well as a description table taken from the Roy et. al. paper you had to read.


<div>
<img src="https://drive.google.com/uc?export=view&id=1L6wwVyCOA3Y2LAKDv5-9peRENmpUAwO9" width="850"/>

<div text-align=center class="caption">EEG-Signal (Source: [5]) </div>
</div>


<div>
<img src="https://drive.google.com/uc?export=view&id=1APK0HDf0znBNd8ogvxy2v0g-j_vhEfry" width="850"/>

<div text-align=center class="caption">EEG-Signal (Source: [5]) </div>
</div>



## Sources
[1] Cohen, Mike X: "Analyzing Neural Time Series Data", MIT Press 2014, Cambridge, Massachusetts

[2] Buzsáki, György; Anastassiou, Costas A; Koch, Christof: "The origin of extracellular fields and currents--EEG, ECoG, LFP and spikes", Nature Review Neuroscience, 2012

[3] Nunez, Paul L.; Srinivasan, Ramesh: "Electric Fields of the Brain: The neurophysics of EEG", Oxford University Press, 2008

[4] Miklody, Daniel: "Theory and Application for Spontaneous EEG", Phd Thesis, TU Berlin, 2015

[5] Roy, Yannick et. al.: Deep learning-based electroencephalography analysis: a systematic review, Journal of Neural Engineering, Volume 16, 2019

In [ ]:
# Install MNE and Braindecode from github
!pip --quiet install git+https://github.com/mne-tools/mne-python.git
!pip --quiet install git+https://github.com/braindecode/braindecode.git
!pip --quiet install moabb

## MNE

In [ ]:
import os
import mne
import copy
import matplotlib.pyplot as plt
import numpy as np

### RAW and Info

The raw object is (probably) the most important data structure of MNE. It is used to hold all the information about the underlying EEG-data as well as the data itself. It provides options for plotting, preprocessing, channel selection and more. Hence, we are interested in how to access data, modify the raw object and eventually visualize the data we have

Usually, raw objects contain a lot of information. However, those informations are not stored directly as attributes but in a dict like object called info. A lot of the information will be redundant for us, however, we want to look at it and get a understanding of the most important information.

[Info Class Documentation](https://mne.tools/stable/auto_tutorials/intro/30_info.html#tut-info-class)

[Raw Class Documentation](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw)

### Loading Data and obtaining information
At first, we will load a sample dataset. With this one at hand, we can check what MNE offers. We will look at some information and briefly discuss how to acces data of the info object

In [ ]:
# MNE contains a sample dataset, that can be used for learning purposes
sample_dir = mne.datasets.sample.data_path()
sample_raw_file = os.path.join(sample_dir, 'MEG', 'sample', 'sample_audvis_raw.fif')

# Load raw object from file path by using IO-routine from MNE
raw = mne.io.read_raw_fif(sample_raw_file)

"""
Remark: 
The .fif file format is usually employed to store eeg data in the context of 
MNE. There are other file formats as well, however, we will stick with this one.
"""

# Before using the info object, we can access some attributes of the raw object directly
n_samples = raw.n_times  # Number of recorded steps in total
time = raw.times         # Time steps in seconds
ch_names = raw.ch_names  # Channel names ... more on this later

# Lets look at the channel names
print(f"Channel names\n"
      f"-------------\n"
      f"{ch_names}\n")

# To get an overview of the information stored in the info object, we can print 
# the keys of the info object
keys = raw.info.keys() 
print(f"Info Keys\n"
  	  f"---------\n"
      f"{keys}\n")

# If we know the keys, we can easily access the values stored within Info, by 
# dictionary style indexing
sampling_frequency = raw.info["sfreq"]
print(f"Sampling Frequency: {sampling_frequency}\n")

# As we have seen above, the channel names does not fit into our scheme we saw
# in the introductory text - we can rename the channels if we want. To see the effect, 
# we first pick the eog channel and rename it afterwards
raw_cp = copy.deepcopy(raw)
raw_cp.rename_channels({'EOG 061': 'blink_detector'})
raw_cp.pick_types(eeg=False, meg=False, eog=True)
print(raw_cp.ch_names)
raw_cp.rename_channels({'blink_detector': 'EOG 061'})
print(raw_cp.ch_names)
raw_cp.pick_types(meg=True, eeg=True, eog=True)
print(raw_cp.ch_names)
print(raw.ch_names)

### Accessing Data / Modifying the Raw Object
Now, that we have seen whats inside of the Raw Object, we want to inspect the data itself. Therefore, the raw object offers different options. The most straightforward one is to index directly as when you are using a numpy array (or torch / tf tensors...). The main difference here is, that a tuple is returned and that, if you index like [0, a:b] you will get the eeg signal with shape [1, ...] and not as a 1-d array. We will also see other ways to obtain the data from the raw object. Depending on the data, you can also preload the data before instantiating the raw object, but this may cause memory issues. 

Besides obtaining the data, one can manipulate the raw object. Maybe, we are only interested in certain channels. Assume we have, for example, a task where the auditory part of the cortex is activ (or we assume to know that). Then we can select channels via pick_channels() that are close to the auditory cortex. Other types of manipulation are interpolation of bad EEG-channels (bad signal-to-noise ratio) or frequency filters. 



In [ ]:
# Now we want to access the data - this can be done in different ways
print("Accessing data in the object\n"
  	  "----------------------------\n")

# Indices to gather in between can be calculated by choosing time in seconds and 
# multiplying by the sampling_frequency ...
tmin = int(1 * sampling_frequency)
tmax = int(5 * sampling_frequency) 

print(f"Min: {tmin}, Max: {tmax} ")
# ... or by using the time_as_index() method of the raw object
t_int = raw.time_as_index([1, 5])
print(f"Min: {t_int[0]}, Max: {t_int[1]} \n")

# Now we can index directly into the raw object
## (1) Gather data via numpy style indexing
channel_idx = 0
eeg_idx, time = raw[channel_idx, tmin:tmax]
print(f"Shape of data, gathered with idx: {eeg_idx.shape}")

## (2) Gather data via channel names ( obtining the value to a key in a dict)
name = ch_names[0]
eeg_name, time = raw[name]
print(f"Shape of data, gathered with name: {eeg_name.shape}")
## Remark: we can also pick the channels of interest directly and remove all the other ones
## by using pick_channels(), where we pass a list of the channel names we are interested in!

## (3) Gather da via channel type (eeg, meg, eog ...)
eeg_ids = mne.pick_types(raw.info, meg=False, eeg=True)
eeg_type, times = raw[eeg_ids]
print(f"Shape of data, gathered with type: {eeg_type.shape}\n")

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(24, 5), sharey=False, sharex=False)
axs[0].plot(time[tmin:tmax], eeg_idx.squeeze(0), label="Idx gathered")
axs[1].plot(time[tmin:tmax], eeg_name.squeeze(0)[tmin:tmax], label="Name gathered")
axs[2].plot(time[tmin:tmax], eeg_type[0, tmin:tmax], label="Type gathered")
for ax in axs:
	ax.set_xlabel("Time [s]")
	ax.set_ylabel("Volatage [V]")
 
plt.show()

## (4) If you just want the data in numpy format, use the getter 
data = raw.get_data()
print(f"{data.shape}")

# You can also inspect the data with an inbuild MNE functionality 
raw.plot(show_scrollbars=False, show_scalebars=False)

print()

## Braindecode

Braindecode was initially written by people from the team of Frank Hutter in Freiburg. Now, other people are involved as well and the codebase is increasing. Braindecode handles the data based on the raw data structure of MNE. This makes life particular easy, since all the precoded transformations are available. We will take a look at this now. Therefore, we loaded braindecode and MOABB from github via pip (see above again). We will import some of the modules and can start. 

### Datasets
Braindecode provides different options for loading datasets. You can either load your own numpy file or a MNE dataset, make use of a MOABB dataset or use one of the ready implemented datasets from Braindecode. While numpy and MNE need some extra work, loading MOABB datasets or Braindecode datasets is straightforward. With the little prior knowledge you have now, you can easily follow the tutorials on the braindecode page to create your own dataset from numpy or MNE. You can follow the API and the tutorials here:
- [Datasets API](https://braindecode.org/api.html#datasets)
- [Datasets from MNE](https://braindecode.org/auto_examples/plot_mne_dataset_example.html): 
- [Datasets from Numpy](https://braindecode.org/auto_examples/plot_custom_dataset_example.html)

[MOABB](http://moabb.neurotechx.com/docs/index.html) (**M**other of **a**ll **B**CI **B**enchmarks) provides open source datasets for EEG in a compact and accessible way. The main goal is, to increase reproducibility in the Brain-Computer-Interface community. The same holds for the field of machine learning and deep learning, where reproducibility is as important as in the pure medical case. Accordingly, the combination of both increases the necessity of reproducible results.

To **load a dataset from MOABB with Braindecode**, we have to choose a dataset from the MOABB page. We will use the *BNCI 2014-001 Motor Imagery dataset*, where the subjects (persons) had to imagine the movement of either the left hand, the right hand, both feet or the tongue. You can find more information about the datasets on the MOABB page by clicking on the specific dataset, e.g. [here](http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014001.html#moabb.datasets.BNCI2014001) for the one we will use. There you can find a short description of the dataset as well as a link to the corresponding paper. The paper provides additional information, e.g. on the montage that was used. This may be important if you want to use the electrode positions as input. 



The **MOABBDataset** class from Braindecode **serves as a wrapper**. This avoids all the boilerplate code we would need to fetch the data from the website. As arguments, we have to pass the name of the dataset we are interested in and a list with the subject ids. In the latter case, the default value is None and will result in loading the whole set. Be careful here, EEG datasets are big and you may run out of memory for very large sets. For demonstration, we will use only subject 1. As an additional input, you can give a dictionary called dataset_kwargs. There you can specify additional information, that can be given to the MOABB dataset (see the API of MOABB for further information).

In [ ]:
# We can import datasets from braindecode 
from braindecode.datasets.moabb import MOABBDataset
from braindecode.datasets import SleepPhysionet

# Preprocessing
from braindecode.preprocessing import \
    (exponential_moving_standardize, preprocess, Preprocessor)

# Events are given, we want windows as an input to the network
from braindecode.preprocessing import create_windows_from_events, create_fixed_length_windows

In [ ]:
# Load the dataset
print("--------------------- Loading Dataset -------------------------")

dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=[1])

# show description of the set
print("\n------------------- Dataset Descirption -----------------------")
print(dataset.description)

"""
We can Iterate through the set. As it is the case for all torch datasets,
the MOABBDataset class overloads the __getitem__() method and hence you can 
index into the dataset as well as iterate through it. 
"""
for x, y in dataset:
  print(f"Channels and steps: {x.shape}, labels: {y}")
  break

"""
As we have seen, MNE provides preprocessing facilities. Luckily, Braindecode
reuses them, s.t. we can apply everything that works in-place on our dataset
"""
print("\n------------------- Apply Preprocessing -----------------------")

# Create list with available preprocessing steps -> Preprocessor module: expects
# the name of the preprocessing step and the corresponding arguments with name (dict type)
preprocessors = [
                 Preprocessor('pick_types', eeg=True, meg=False, stim=True, verbose=False), # Only use eeg and stimulus channels
                 Preprocessor('resample', sfreq=100, verbose=False)  # Downsample
]

# Before preprocessing
print(f"List of channel names wo preprocessing: {dataset.datasets[0].raw.ch_names}," 
      f"Sample frequency wo preprocessing: {dataset.datasets[0].raw.info['sfreq']}")

# Apply preprocessing by inbuild method 
preprocess(dataset, preprocessors)

# After preprocessing
print(f"List of channel names w preprocessing: {dataset.datasets[0].raw.ch_names}," 
      f"Sample frequency w preprocessing: {dataset.datasets[0].raw.info['sfreq']}")


# It is common, to work on windowed data. That is, you select a range of steps
# beginning at t=0 (or later if you want) and create the first window, e.g. 
# data[0:100, :]. With a predefined stride you move forward and the next window 
# is created from data[stride: 100 + stride, :], where the stride could be 
# 1, 10, 100 or anything else suitable. In this way, you do not need to pass 
# the full sequence to the network. This makes sense, since for a long recording, 
# the influence of a event at ti has little influence on time steps further in the
# future. This is something of interest for all of you, so keep this in mind.
windows_ds = create_windows_from_events(
    dataset, trial_start_offset_samples=0, trial_stop_offset_samples=100,
    window_size_samples=400, window_stride_samples=100, drop_last_window=False
)

max = 3
fig, axs = plt.subplots(1, max + 1, figsize=(4. * (max + 1), 4), sharey=True)
for i, (x, y, ix) in enumerate(windows_ds):
    axs[i].plot(x[0:1].T)
    axs[i].set_title(f"Label={y}")
    if i == max:
      break

fig.tight_layout()

### Task 
You will repeat the steps above with another dataset. Your task is to create a MOABB dataset from the Physionet dataset. Therefore, search for the name of the set on the MOABB webpage. Load the first and the third subject only. Plot the channel names. Afterwards, pick the electrodes on the left hemisphere corresponding to the parietal lobe for subject 3. Plot them with the inbuild raw plotting routine of MNE. Afterwards, crop the data from the second to the third second and plot again via MNE. In order to get nicely behaved plots, use the scaling argument of the plot() method and use 150 $\mu V$ as the peak value. 

In [ ]:
# Load the data from MOABB
task_ds = MOABBDataset('PhysionetMI', subject_ids=[1, 3])

# Extract the correct raw
my_raw = task_ds.datasets[1].raw
# Print the channel names
print(my_raw.ch_names)

# Select
my_raw.pick_channels(["C1", "C2", "C3", "C4", "C5", "C6", "FC4", "FC5", "FC6", "FC3", "FC2", "FC1"])

# Plot full
my_raw.plot(show_scalebars=False, show_scrollbars=False, scalings=150e-6)

# Crop a copy raw
my_raw_cp = copy.deepcopy(my_raw)
my_raw_cp.crop(4.2, 8.304)

# Plot the cropped copy
my_raw_cp.plot(show_scalebars=False, show_scrollbars=False, scalings=150e-6)

my_raw_cp.filter(l_freq=6, h_freq=20)
my_raw_cp.plot(show_scalebars=False, show_scrollbars=False, scalings=150e-6)


## Playground

You can inspect what happens, when you apply different preprocessing steps to the MNE RAW datastructure (here the one saved within the braindecode dataset) and play around with them. We did not look into it. However, the usage is straightforward. MNE provides different preprocessing options that ship with the data structure. You can call them as methods of the class, e.g. raw.pick_channel(args). However, for braindecode things are a little bit different. Here, you provide the name of the preprocessing step. Have a look and inspect what is happening, if you change the different settings. 

**Description of the box**

1. Plot settings
  - Plot duration: time to be plotted in seconds
  - Plot sensors: Plots the sensor topology, i.e. the arrangment of sensors on the head
  - Topo plot: The topographic projection of the signal towards the head shape (you have to select the channels by type first - type: eeg)
  - PSD plot: Plots the power spectral density of the signals
2. Pick Channels
  - Pick: Here you can turn the option on and off
  - Pick types: in MNE, you can choose different ways of selecting channels (by idx, by name, by electrode type)
  - Pick boxes: for the names, you can choose one (or more) channel names, for the idx you can use the slider and for the types the type box obviously
3. Bandpass Filtering
  - Filter: yes / no (turn on, off)
  - Freq: By selecting the maximum and the minimum frequency, you can set the filter limits
4. Exponential Standadization
  - The standardization is used to remove small scale fluctuations in the data which can appear e.g. due to noise. By using an exponential approach, sample points in the past have vanishing influence on the smoothed result
5. Windowing
  - Windowing is commonly used to pick certain time spans of interest. This is also useful for deep learning in various ways.

If you are not windowing, you will obtain a power spectral density plot of the signal. The plot tells you, how much power is contained withing the signal for a specific frequency f. Recovering the exact PSD could be an interesting loss for regression tasks. However, the PSD is normaly used to determine the signal-to-noise ratio and to design filters to suppress noise in the signal. So if you have trouble with the concept, this is also fine, as it is likely that you do not need it.

Here, yould try to find specific frequeny patterns in the data. There are common patterns for EEG data, namely
- [alpha waves](https://en.wikipedia.org/wiki/Alpha_wave), frequency range of 8–12 Hz
- [beta waves](https://en.wikipedia.org/wiki/Beta_wave), frequency range of 12.5 and 30 Hz
- [gamma waves](https://en.wikipedia.org/wiki/Gamma_wave), frequency range of 25 and 140 Hz
- [delta waves](https://en.wikipedia.org/wiki/Delta_wave), frequency range of 0.5 and 4 Hz

<div>
<img src="https://drive.google.com/uc?export=view&id=1YbyAYrznF_brVLTeTW6QXcpARW5725YJ" width="700"/>

<div text-align=center class="caption">[Brain Wave Patterns](https://blogs.dickinson.edu/writingsciencenews18/2018/02/alpha-waves-attention-anxiety-oh-my/) </div>
</div>



### Helpers

In [ ]:
def plot_raw_signal(**kwargs):
    # Subject ID
    sbj_id = kwargs["subject_idx"]

    # Empt list to store applied preprocessors
    preprocessors = []

    # Make copy of the instance
    cp_dataset = copy.deepcopy(dataset)

    # suppress output stuff...
    cp_dataset.datasets[sbj_id].raw.verbose = False
    scale = 'auto'

    # Check for preprocessors 
    if kwargs['filter'] is True:
        preprocessors.append(
            Preprocessor('filter',
                         l_freq=kwargs['freq_range'][0],
                         h_freq=kwargs['freq_range'][1],
                         verbose=False
                         )
        )

    if kwargs['smooth'] is True:
        preprocessors.append(
            Preprocessor(exponential_moving_standardize,
                         init_block_size=kwargs['init_block_size'],
                         factor_new=kwargs['factor']
                         )
        )
        scale = 0.2

    if kwargs['pick'] is True:
        if kwargs['pick_types'] == "Types":
            picks = kwargs['ch_types']
            print(picks)
            pick_dict = dict(zip(picks, [True for _ in picks]))
            display(pick_dict)
            preprocessors.append(Preprocessor('pick_types', **pick_dict))
        elif kwargs['pick_types'] == "Names":
            picks = kwargs['ch_names']
            preprocessors.append(Preprocessor('pick_channels', ch_names=picks))
        else:
            picks = list(range(kwargs["ch_ids"][0], kwargs["ch_ids"][1]))
            preprocessors.append(Preprocessor('pick', picks=picks))

    # if any preprocessors, apply
    if len(preprocessors) > 0:
        preprocess(cp_dataset, preprocessors=preprocessors)

    if kwargs["window"] and kwargs["crop"]:
      raise ValueError("Windowing and cropping at the same time not allowed!")

    if kwargs["crop"] is True:
      cp_dataset.datasets[sbj_id].raw.crop(kwargs["crop_t_min"], kwargs["crop_t_max"])

    # if one wants to window the output (changes output from mne plot to plt plot)
    if kwargs["window"] is True:
        windows_to_plot = 4
        windows_dataset = create_windows_from_events(
            cp_dataset, 
            trial_start_offset_samples=kwargs["offset_begin"],
            trial_stop_offset_samples=kwargs["offset_stop"], 
            window_size_samples=kwargs["window_size"], 
            window_stride_samples=kwargs["window_stride"], 
            )
        
        fig, axs = plt.subplots(1, windows_to_plot, 
                                  figsize=(4*windows_to_plot, 4), 
                                  sharey=True
                                  )
        
        for i, (x, y, window_id) in enumerate(windows_dataset):
          if i == windows_to_plot:
            break
        
          axs[i].plot(x.T)
          axs[i].legend([f"Window {i}"])
          axs[i].set_title(f"Label: {y}")
          axs[i].grid(True)

        fig.tight_layout()
    
    else:
      print("Raw EEG Plot\n"
            "------------")
      if kwargs["pick"] is False:
        cp_dataset.datasets[sbj_id].raw.plot(duration=kwargs['duration'], scalings='auto', show_scalebars=False, show_scrollbars=False)
      else:
        cp_dataset.datasets[sbj_id].raw.plot(duration=kwargs['duration'], scalings=50e-6, show_scalebars=False, show_scrollbars=False)
        
      if kwargs["plot_psd"]:
        print("\nRaw PSD Plot\n"
              "--------------")
        cp_dataset.datasets[sbj_id].raw.plot_psd()
      if kwargs["plot_sp"]:
        print("\nRaw Sensor Position Plot\n"
              "--------------------------")        
        cp_dataset.datasets[sbj_id].raw.plot_sensors()
      if kwargs["plot_topo"]:
        print("\nRaw Topomap Plot\n"
              "------------------")
        data = cp_dataset.datasets[sbj_id].raw.get_data()[:, 0]
        print(data.shape)
        info = cp_dataset.datasets[sbj_id].raw.info
        mne.viz.plot_topomap(data=data, pos=info)


In [ ]:
import copy

# Widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

"""
WIDGETS FOR PREPROCESSING
"""
# ---------------------------------- FILTER ---------------------------------- #
pp_filter = widgets.Checkbox(value=False, description="Filter?: y/n")
freq_range = widgets.IntRangeSlider(min=0, max=49, value=(0, 49), 
                                    description="Freq. range to maintain", 
                                    continuous_update=False
                                    )
filter_box = widgets.HBox([pp_filter, freq_range])


# -------------------------------- SMOOTHING --------------------------------- #
pp_smoothing = widgets.Checkbox(value=False, description="Smooth?: y/n")
init_block_size = widgets.IntSlider(min=500, max=1500, step=100, value=1000, 
                                    description="nSteps", disabled=False)
factor = widgets.FloatSlider(min=0.001, max=0.01, step=0.001, value=0.001, 
                             description='Safety Factor:', disabled=False, 
                             continuous_update=False, orientation='horizontal', 
                             readout=True, readout_format='.3f')
smoothing_box = widgets.HBox([pp_smoothing, init_block_size, factor])

# ---------------------------------- PICKS ----------------------------------- #
pp_pick = widgets.Checkbox(value=False, description="Pick?: y/n")
pp_pick_type = widgets.Select(options=["Types", "Names", "Channel Ids"], 
                              description="Pick types")
by_ch_id = widgets.IntRangeSlider(min=0, max=22, step=1, value=(0, 22), 
                                  description="Channel ids to pick")
by_name = widgets.SelectMultiple(options=dataset.datasets[0].raw.ch_names, 
                                 value=[dataset.datasets[0].raw.ch_names[0]], 
                                 description='Channel names'
                                 )
by_type = widgets.SelectMultiple(options=list(set(dataset.datasets[0].raw.get_channel_types())), 
                                 value=list(set(dataset.datasets[0].raw.get_channel_types())), 
                                 description='Channel types')
pick_box = widgets.HBox([pp_pick, pp_pick_type])
by_box = widgets.HBox([by_ch_id, by_name, by_type])
pick_grid = widgets.VBox([pick_box, by_box])

# ----------------------- Windowing and Cropping ----------------------------- #
pp_window = widgets.Checkbox(value=False, description="Window data?: y/n")

window_size_slider = widgets.IntSlider(min=100, max=1000, step=100, value=400, 
                                       continuous_update=False, 
                                       description="Window Size"
                                       )

window_stride_slider = widgets.IntSlider(min=1, max=20, step=1, value=1, 
                                         continuous_update=False, 
                                         description="Stride btw. windows"
                                         )

trial_start_offset_slider = widgets.IntSlider(min=0, max=10, value=0, step=1, 
                                              continuous_update=False, 
                                              description="Start offset"
                                              )
trial_stop_offset_slider = widgets.IntSlider(min=50, max=150, value=100, step=10, 
                                             continuous_update=False, 
                                             description="Stop offset"
                                             )

window_box = widgets.HBox(children=[pp_window, window_size_slider, window_stride_slider, 
                                       trial_start_offset_slider, trial_stop_offset_slider
                                      ]
                             )

pp_crop = widgets.Checkbox(value=False, description="Crop: y/n?")
crop_t_min = widgets.BoundedFloatText(value=0, min=0, max=dataset.datasets[0].raw.times[-2], description="t0")
crop_t_max = widgets.BoundedFloatText(value=0, min=dataset.datasets[0].raw.times[1], max=dataset.datasets[0].raw.times[-1], description="tmax")
crop_box = widgets.HBox(children=[pp_crop, crop_t_min, crop_t_max])

cut_box = widgets.VBox(children=[window_box, crop_box])

# ------------------------------ PLOT SETTINGS ------------------------------- #
subject_select = widgets.Select(options=list(range(0, len(dataset.datasets))), value=0, description="Subject")
duration_slider = widgets.IntSlider(min=1, max=10, value=10, continuous_update=False,
                                    description="Duration of the plotted signal"
                                    )
sensor_pos_box = widgets.Checkbox(value=False, description="Sensor Plot?")
psd_box = widgets.Checkbox(value=False, description="PSD Plot")
topo_box = widgets.Checkbox(value=False, description="Topo Plot")
topo_slider = widgets.BoundedIntText(value=0, min=0, max=dataset.datasets[0].raw.n_times - 1)
v_box = widgets.VBox([topo_box, topo_slider])
plot_box = widgets.HBox([subject_select, duration_slider, sensor_pos_box, topo_box, psd_box])


# --------------------------------- Assembly --------------------------------- #

children = [plot_box, pick_grid, smoothing_box, filter_box, cut_box]
titles = ["Plot Settings", "Pick Channels", "Exponential Standardization", "Bandpass Filtering", "Cutting Utils"]
pp_ui = widgets.Accordion(children=children)

for index, title in enumerate(titles):
  pp_ui.set_title(index, title=title)

### Run for Fun

In [ ]:
%matplotlib inline
"""plot_settings = {"duaration": duration_slider, "plot_sp": sensor_pos_box}
pick_channels = {"pick": pp_pick, 'pick_types': pp_pick_type, 
                 'ch_ids': by_ch_id, 'ch_name':}"""
out = widgets.interactive_output(
    plot_raw_signal, {"subject_idx": subject_select, 'pick': pp_pick, 'smooth': pp_smoothing, 'filter': pp_filter, 
        'freq_range': freq_range, 'init_block_size': init_block_size, 
        'factor': factor, 'pick_types': pp_pick_type,
        'ch_ids': by_ch_id, 'ch_names': by_name, 'ch_types': by_type, 
        'duration': duration_slider, "window_size": window_size_slider, 
        "window_stride": window_stride_slider, "window": pp_window,
        "offset_begin": trial_start_offset_slider,
        "offset_stop": trial_stop_offset_slider,
        "crop": pp_crop, "crop_t_min": crop_t_min, "crop_t_max": crop_t_max,
        "plot_sp": sensor_pos_box, "plot_psd": psd_box, "plot_topo": topo_box
        }
    )



print(dataset.datasets[0].raw.ch_names)

display(pp_ui, out)